In [ ]:
#install the build your own latent pip package
!pip install byol-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 46.9 MB/s eta 0:00:00


In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Code mainly copied from the byol github documentation at https://github.com/lucidrains/byol-pytorch

import torch
from byol_pytorch import BYOL
from torchvision import models
import os
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

resnet = models.resnet50(pretrained=True)

learner = BYOL(
    resnet,
    image_size = 224,
    hidden_layer = 'avgpool'
)

opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

def sample_unlabelled_images():
######################### Custom Code ##########################################
    image_directory = '/content/drive/MyDrive/backboneData'

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    image_tensors = []

    for filename in tqdm(os.listdir(image_directory), desc='iterating images'):
        img_path = os.path.join(image_directory, filename)
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)
        image_tensors.append(img_tensor)

    return torch.stack(image_tensors)
################################################################################
for _ in tqdm(range(60), desc='iterating range'):
    images = sample_unlabelled_images()
    loss = learner(images)
    opt.zero_grad()
    loss.backward()
    opt.step()
    learner.update_moving_average()

######################### Custom Code ##########################################
torch.save(resnet.state_dict(), '/content/drive/MyDrive/backbones/improved-net.pth')
################################################################################


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 196MB/s]
iterating range: 100%|██████████| 60/60 [1:23:57<00:00, 83.96s/it]
